<a href="https://colab.research.google.com/github/Amarantine-xiv/Another-FF14-Combat-Sim/blob/main/CoreSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2023 A. Falena

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
VERSION=0.05

In [2]:
#@title Imports

import numpy as np
import matplotlib.pyplot as plt
import time
import copy
from dataclasses import dataclass
import heapq
from enum import Enum

In [3]:
#@title Game Constants
@dataclass(frozen=True)
class GameConsts:
  GCD_RECAST_TIME: float = 2500
  DAMAGE_SNAPSHOT_TIME_BEFORE_CAST_FINISHES = 500
  # level mods from akhmorning
  LEVEL_DIVS = {90: 1900}
  LEVEL_SUBS= {90: 400}
  SPEED_COEFFICIENT=130

In [4]:
#@title StatFns, Stats
class StatFns:
  @staticmethod
  def get_time_using_speed_stat(t_ms, speed_stat, level=90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]

    tmp = np.ceil(GameConsts.SPEED_COEFFICIENT*(level_sub-speed_stat)/level_div)
    tmp2 = t_ms*(1000+tmp)/10000
    tmp3 = np.floor(tmp2)/100
    return int(1000*tmp3)

@dataclass(frozen=True)
class Stats():
  wd: float
  weapon_delay: float
  main_stat: float
  det_stat: float
  dh_stat: float
  crit_stat: float
  speed_stat: float
  job_class: str
  tenacity: float = None
  num_roles_in_party: float = 5
  healer_or_caster_strength: float= None
  def __post_init__(self):
    pass
     #object.__setattr__(self, 'job_mod', JobClassFns.JOB_MODS[self.job_class])
     #object.__setattr__(self, 'trait_dmg_mult', JobClassFns.compute_trait_dmg_mult(self.job_class))

In [5]:
#@title Test utils

class TestClass:
  def __init__(self):
    self.test_fns = []

  def is_a_test(f):
    f.__is_a_test__ = True
    return f

  def _get_test_methods(self):
    test_methods = []
    for fn_name in dir(self):
      fn = getattr(self, fn_name)
      if getattr(fn, '__is_a_test__', False):
        test_methods.append(fn)
    return test_methods

  def _print_result(self, passing, failing):
    print('Testing for {}. {}/{} tests passed.'.format(self.__class__.__name__, len(passing), len(passing)+len(failing)))
    if len(failing) > 0:
      print('Failing tests:')
      for test_info in failing:
        print("{}: {}\n".format(test_info[0], test_info[1]))
    print('Passing tests:')
    for test_name in passing:
      print("{}".format(test_name))

  def run_single(self, test_name):
    for test_fn in self._get_test_methods():
      if test_name == test_fn.__name__:
        test_passed, err_msg = test_fn()
        if test_passed:
          print('{} passed!'.format(test_name))
          return
        else:
          print('{} failed: {}'.format(test_name, err_msg))
          return
    print('No test found with name {}'.format(test_name))

  def run_all(self):
    passing = []
    failing = []
    for test_fn in self._get_test_methods():
      test_name = test_fn.__name__
      test_passed, err_msg = test_fn()
      if (test_passed):
        passing.append(test_name)
      else:
        failing.append((test_name, err_msg))
    self._print_result(passing, failing)
    print("================")


In [6]:
#@title TimingSpec

@dataclass(frozen=True)
class TimingSpec:
  base_cast_time: int
  is_GCD: bool
  gcd_base_recast_time: int = None #after you use this skill, how long until you can use a gcd?
  base_recast_time: int = 0 # the base "cooldown" of the skill- when can you use it next?
  animation_lock: int = 65 #does not take into account ping
  application_delay: int = 0 #how long after the cast finishes does the skill get applied

  def __error_check(self):
    #This is such a common error, that we need to defend against it.
    assert isinstance(self.base_cast_time, int), "Base cast time should be an int in ms. Did you put it in seconds?"
    assert isinstance(self.application_delay, int), "Application delay should be an int in ms. Did you put it in seconds?"

  def __post_init__(self):
    if self.gcd_base_recast_time is None:
      if self.is_GCD:
        # assume a default of 2500ms recast time for all gcds, unless otherwise stated
        object.__setattr__(self, 'gcd_base_recast_time', GameConsts.GCD_RECAST_TIME)
      else:
        # By default, non-gcds don't affect gcd_base_recast_time. If this is not
        # true for this skill, then set gcd_base_recast_time explicitly.
        object.__setattr__(self, 'gcd_base_recast_time', 0)
    self.__error_check()

  def __str__(self):
    res = "  Cast time: {}\n  is_GCD: {}".format(self.cast_time, self.is_GCD)
    return res

In [7]:
#@title Skill, SkillLibrary

@dataclass(frozen=True)
class Skill:
  # Note: All fields must be either 1) a primitive, 2) a class with the dataclass
  # decorator, or 3) defined with an appropriate __eq__ function.
  name: str
  timing_spec: TimingSpec = None

  def __str__(self):
    res = "---Skill name: {}---\n".format(self.name)
    res += "TimingSpec:\n{}\n".format(str(self.timing_spec))
    return res

class SkillLibrary:
  def __init__(self):
    self._skills = {}

  def get_skill(self, skill_name, job_class):
    return self._skills[job_class][skill_name]

  def add_job_class(self, job_name):
    self._skills[job_name] = {}

  def add_skill(self, skill, job_class):
    skill_name = skill.name
    if skill_name in self._skills[job_class]:
      raise RuntimeError('Duplicate skill being added to the skill library (this is probably a naming error). Job: {}, Skill name: {}'.format(job_class, skill_name))
    self._skills[job_class][skill.name] = skill

  def print_skills(self):
    for job_name in self._skills:
      for skill_name in self._skills[job_name]:
        print("Job name: {}, Skill name: {}".format(job_name, skill_name))

In [8]:
#@title SkillLibrary population
all_skills = SkillLibrary()
all_skills.add_job_class('test_job')

OGCD_INSTANT = TimingSpec(base_cast_time=0, is_GCD=False)
GCD_2500 = TimingSpec(base_cast_time=2500, is_GCD=True)

test_gcd = Skill(name='test_gcd', timing_spec = GCD_2500)
test_ogcd = Skill(name='test_ogcd', timing_spec = OGCD_INSTANT)
all_skills.add_skill(test_gcd, 'test_job')
all_skills.add_skill(test_ogcd, 'test_job')

In [9]:
#@title RotationBuilder

class RotationBuilder:

  """ A utility class used to turn 1) a series of button (skills) pressed and optionally the specific time they were pressed, and 2) proc application times into a series of damage instances and applicable buffs/debuffs."""
  def __init__(self, stats, skill_library):
    self._stats = stats
    self._skill_library = skill_library
    self._q = []
    self._q_timed = []

  @staticmethod
  def _print_q(q):
    q.sort(key=lambda x: x[0])
    for (time, skill) in q:
      print('{}: {}'.format(time, skill.name))

  def add_to_rotation(self, skill_name, t=None):
    """Adds a skill to be used. All skills are considered to be used sequential at the first available opportunity, unless a time t is specified."""
    """ Time is assumed to be in seconds"""
    skill = self._skill_library.get_skill(skill_name, self._stats.job_class)
    if t is None:
      heapq.heappush(self._q, skill)
    else:
      heapq.heappush(self._q_timed, (int(1000*t), skill))

  def __get_application_time(self, t, skill):
    return t + skill.timing_spec.cast_time + skill.timing_spec.application_delay

  # Result: (snapshot time, application time, skill, snapshots_buffs, snapshots_debuffs), indicating
  # whether buffs and debuffs snapshot at the given given time for that skill
  # snapshot_time.
  def _get_skill_timing(self):
    q_timed_snapshot = []
    q = copy.deepcopy(self._q_timed)
    while len(q) > 0:
      (t, skill) = heapq.heappop(q)
      cast_time = StatFns.get_time_using_speed_stat(skill.timing_spec.base_cast_time, self._stats.speed_stat)
      snapshot_time = t + max(0, cast_time-500)
      application_time = t + cast_time + skill.timing_spec.application_delay
      heapq.heappush(q_timed_snapshot, (snapshot_time, application_time, skill, True, True))
    return q_timed_snapshot

  def print_sequential_rotation(self):
    q = copy.deepcopy(self._q_timed)
    RotationBuilder._print_q(q)

In [10]:
class RotationBuilderUtilTest(TestClass):
  def __setup_test_skill_library(self):
    self._all_skills = SkillLibrary()
    self._all_skills.add_job_class(self.TEST_JOB_NAME)

    gcd_2500 = TimingSpec(base_cast_time=2500, is_GCD=True)
    gcd_2500_app_delay = TimingSpec(base_cast_time=2500, application_delay=100, is_GCD=True)
    ogcd_instant = TimingSpec(base_cast_time=0, is_GCD=False)

    test_gcd = Skill(name=self.TEST_GCD_NAME, timing_spec = gcd_2500)
    test_gcd_app_delay = Skill(name=self.TEST_GCD_APP_DELAY_NAME, timing_spec = gcd_2500_app_delay)
    test_ogcd = Skill(name=self.TEST_OGCD_NAME, timing_spec = ogcd_instant)

    self._all_skills.add_skill(test_gcd, self.TEST_JOB_NAME)
    self._all_skills.add_skill(test_ogcd, self.TEST_JOB_NAME)
    self._all_skills.add_skill(test_gcd_app_delay, self.TEST_JOB_NAME)

  def __init__(self):
    self.TEST_JOB_NAME = 'test_job'
    self.TEST_GCD_NAME = 'test_gcd'
    self.TEST_OGCD_NAME = 'test_ogcd'
    self.TEST_GCD_APP_DELAY_NAME = 'test_gcd_app_delay'
    self.__setup_test_skill_library()

  @TestClass.is_a_test
  def skill_timing_test_no_buffs_no_dot(self):
    test_passed = True
    err_msg=""

    # For a 2500 ms gcd, this spell speed should result in a 2440 sm (2.44s) GCD.
    stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = self.TEST_JOB_NAME)
    rb = RotationBuilder(stats, self._all_skills)
    rb.add_to_rotation(self.TEST_GCD_NAME, 0)
    rb.add_to_rotation(self.TEST_GCD_NAME, 3.5)
    rb.add_to_rotation(self.TEST_OGCD_NAME, 2.4)
    rb.add_to_rotation(self.TEST_GCD_APP_DELAY_NAME, 4.0)
    expected = [(1940, 2440, self._all_skills.get_skill(self.TEST_GCD_NAME, self.TEST_JOB_NAME), True, True),
                (2400, 2400, self._all_skills.get_skill(self.TEST_OGCD_NAME, self.TEST_JOB_NAME), True, True),
                (3500+1940, 3500+2440, self._all_skills.get_skill(self.TEST_GCD_NAME, self.TEST_JOB_NAME), True, True),
                (4000+1940, 4000+2440+100, self._all_skills.get_skill(self.TEST_GCD_APP_DELAY_NAME, self.TEST_JOB_NAME), True, True)]

    result = rb._get_skill_timing()
    if len(expected) != len(result):
      test_passed = False
      err_msg += "Expected 3 skills returned. Instead got {}. ".format(len(result))
    for i in range(0, len(expected)):
      if expected[i] != result[i]:
        test_passed = False
        err_msg += "Position {} was not the same.\n Expected: {}\n Actual: {}".format(i, expected[i], result[i])

    return test_passed, err_msg

In [11]:
#@title TestClasses

class TimingSpecTest(TestClass):
  @TestClass.is_a_test
  def gcd_override_gcd_test(self):
    test_passed = True
    err_msg=""
    cast_spec = TimingSpec(base_cast_time=2000, is_GCD=True)
    if cast_spec.gcd_base_recast_time != 2500:
      err_msg = "Recast time was expected to be 2500, but it was {}.".format(cast_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

  @TestClass.is_a_test
  def gcd_override_ogcd_test(self):
    test_passed = True
    err_msg=""
    cast_spec = TimingSpec(base_cast_time=2000, is_GCD=False)
    if cast_spec.gcd_base_recast_time != 0:
      err_msg = "Recast time was expected to be 0, but it was {}.".format(cast_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

  @TestClass.is_a_test
  def gcd_override_set_recast_time(self):
    test_passed = True
    err_msg=""
    cast_spec = TimingSpec(base_cast_time=2000, gcd_base_recast_time=1500, base_recast_time=60000, is_GCD=False)
    if cast_spec.gcd_base_recast_time != 1500:
      err_msg = "Recast time was expected to be 1500, but it was {}.".format(cast_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

class StatFnsTest(TestClass):
  @TestClass.is_a_test
  def get_time_using_speed_stat_test(self):
    test_passed = True
    err_msg=""

    gcd_inputs= {2500: ([1408, 1409, 1410], [2330, 2320, 2320]),
                  3500: ([999, 1000, 1001], [3360, 3350, 3350])}

    for gcd_time, inputs_and_outputs in gcd_inputs.items():
      speed_stat = inputs_and_outputs[0]
      expected = inputs_and_outputs[1]
      for i in range(0, len(speed_stat)):
          gcd_actual = StatFns.get_time_using_speed_stat(gcd_time, speed_stat[i])
          gcd_expected = expected[i]
          if gcd_actual != gcd_expected:
            err_msg += "gcd_time expected: {}. Actual: {}. For 3.5 gcd. ".format(gcd_expected, gcd_actual)
            test_passed = False

    return test_passed, err_msg

In [12]:
TimingSpecTest().run_all()
StatFnsTest().run_all()
RotationBuilderUtilTest().run_all()

Testing for TimingSpecTest. 3/3 tests passed.
Passing tests:
gcd_override_gcd_test
gcd_override_ogcd_test
gcd_override_set_recast_time
Testing for StatFnsTest. 1/1 tests passed.
Passing tests:
get_time_using_speed_stat_test
Testing for RotationBuilderUtilTest. 1/1 tests passed.
Passing tests:
skill_timing_test_no_buffs_no_dot
